In [1]:
import os
import sys
sys.path.append('../')

# from src.model.KLIEP import *
import numpy as np
import torch
import torch.nn as nn

import argparse
import json

from copy import deepcopy

import torch.nn as nn

from src.simulator.utils import generate_model

In [2]:
def load_config(config_path, proj_name):
    PATH = os.path.join(config_path, f"config_{proj_name}.json")
    with open(PATH, 'r') as f:
        config = json.load(f)
    return config

def load_model(model_name, modelPATH, config, device):
    # device = torch.device('cpu')
    PATH = os.path.join(modelPATH, model_name)
    model_dict = torch.load(PATH, map_location=device)

    global_model = generate_model(config).to(device)
    
    if (config.agg_method != "Center") and (config.agg_method != "Local"):
        global_model.load_state_dict(model_dict['global_model'], strict=False)
        local_model_dict = model_dict['local_model']
        
    else:
        global_model.load_state_dict(model_dict['model'], strict=False)
        local_model_dict = None

    return global_model, local_model_dict

def load_model(model_name, modelPATH, config, device):
    # device = torch.device('cpu')
    PATH = os.path.join(modelPATH, model_name)
    model_dict = torch.load(PATH, map_location=device)

    global_model = generate_model(config).to(device)
    
    if (config.agg_method != "Center") and (config.agg_method != "Local"):
        global_model.load_state_dict(model_dict['global_model'], strict=False)
        local_model_dict = model_dict['local_model']
        
    else:
        global_model.load_state_dict(model_dict['model'], strict=False)
        local_model_dict = None

    return global_model, local_model_dict

In [ ]:
batch_size =16
source_data = torch.tensor(np.random.normal(0, 1, (batch_size, 96, 128, 96)), dtype=torch.float32)
source_data = source_data.unsqueeze(1)
print(source_data.shape)

In [4]:
basePATH = "/NFS/Users/moonsh/AdaptFL/ckpt/"

In [38]:
ckptPATH = os.path.join(basePATH, "Center", "dandy-shadow-1")
config = load_config(ckptPATH, "dandy-shadow-1")
config['batch_size'] = 32
config['num_workers'] = 2
config['nowandb'] = True
config = argparse.Namespace(**config)

model_name = f'Center_best_model_dandy-shadow-1.pth'
local_model = load_model(model_name, ckptPATH, config, torch.device('cpu'))
local_model = local_model[0]

In [40]:
wandb_name = "fresh-feather-2"

ckptPATH = os.path.join(basePATH, "FedAvg", wandb_name)
config = load_config(ckptPATH, wandb_name)
config['batch_size'] = 32
config['num_workers'] = 2
config['nowandb'] = True
config = argparse.Namespace(**config)

model_name = f'fresh-feather-2_best_round_050.pth'
glob_model = load_model(model_name, ckptPATH, config, torch.device('cpu'))
local_weight = glob_model[1][5]

glob_model = glob_model[0]
# local_model = deepcopy(glob_model)
# local_model.load_state_dict(local_model, strict=False)

In [ ]:
local_model = deepcopy(glob_model)
local_model.load_state_dict(local_weight, strict=False)

In [42]:
activation={'Local': {},
            'Global': {}}

In [43]:
def get_activation(model_name, layer_name):
    def hook(module, input, output):
        activation[model_name][layer_name] = output.detach()
    return hook


def register_hooks_for_model(model, model_name):
    hooks = []
    for name, layer in model.named_modules():
        if (isinstance(layer, nn.Conv3d) or isinstance(layer, nn.Linear)) and ('downsample' not in name):
            hook = layer.register_forward_hook(get_activation(model_name, name))
            hooks.append(hook)
    return hooks

def remove_hooks(hooks):
    for hook in hooks:
        hook.remove()

In [44]:
hooks_model_a = register_hooks_for_model(local_model, 'Local')
hooks_model_b = register_hooks_for_model(glob_model, 'Global')

In [45]:
loc_rep = local_model(source_data) # predict 값
glob_rep = glob_model(source_data) # predict 값

In [ ]:
num_layers = 0
for (layer_name, loc_output), glob_output in zip(activation['Local'].items(), activation['Global'].values()):
    print(layer_name)
    if "layer3.0." in layer_name:
        break
    num_layers += 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

activations = loc_output
activations2 = glob_output

# Flatten activations for a single channel
for i in range(233, 235):
    channel_activations = activations[:, i, :, :, :].flatten()
    channel_activations2 = activations2[:, i, :, :, :].flatten()

    # Compute histogram
    # hist, bin_edges = np.histogram(channel_activations, bins=100, density=True)

    # # Plot histogram
    # plt.hist(channel_activations, bins=100, density=True)
    # plt.title('Probability Distribution of Activations (Channel 0)')
    # plt.xlabel('Activation Value')
    # plt.ylabel('Probability Density')

    # Kernel Density Estimation
    kde = gaussian_kde(channel_activations)
    activation_range = np.linspace(channel_activations.min(), channel_activations.max(), 1000)
    kde_values = kde(activation_range)

    kde2 = gaussian_kde(channel_activations2)
    activation_range2 = np.linspace(channel_activations2.min(), channel_activations2.max(), 1000)
    kde_values2 = kde2(activation_range2)


    # Plot KDE
    plt.plot(activation_range, kde_values, color='red')
    plt.plot(activation_range2, kde_values2, color='blue')
    plt.title('KDE of Activations (Channel 0)')
    plt.xlabel('Activation Value')
    plt.ylabel('Probability Density')
    plt.show()

In [18]:
remove_hooks(hooks_model_a)
remove_hooks(hooks_model_b)